### Задание

Взять тот же датасет, который был на вебинаре и предобученную модель для задачи суммаризации
1. Проверить насколько хорошо она суммаризирует
2. (дополнительно) Сделать генерацию заголовков для статьи (обучить модель для генерации заголовков)

In [1]:
!pip install transformers sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.4 MB/s 
     |████████████████████████████████| 1.3 MB 46.6 MB/s 
     |████████████████████████████████| 182 kB 61.6 MB/s 
     |████████████████████████████████| 7.6 MB 58.0 MB/s 


In [2]:
!pip3 install torch torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!wget -q https://www.dropbox.com/s/43l702z5a5i2w8j/gazeta_train.txt 
!wget -q https://www.dropbox.com/s/k2egt3sug0hb185/gazeta_val.txt 
!wget -q https://www.dropbox.com/s/3gki5n5djs9w0v6/gazeta_test.txt

In [4]:
import json
import random
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [5]:
def read_gazeta_records(file_name, shuffle=True, sort_by_date=False):
    assert shuffle != sort_by_date
    records = []
    with open(file_name, "r") as r:
        for line in r:
            records.append(json.loads(line))
    if sort_by_date:
        records.sort(key=lambda x: x["date"])
    if shuffle:
        random.shuffle
    return records

def summurize(rec):
    article_text = ("{}".format(test_records[rec]["text"]))
    input_ids = tokenizer([HANDLER(article_text)], 
                          return_tensors="pt", 
                          padding="max_length", 
                          truncation=True, 
                          max_length=512)["input_ids"]
    output_ids = model.generate(input_ids=input_ids,
                                max_length=84,
                                no_repeat_ngram_size=2,
                                num_beams=4)[0]
    summary = tokenizer.decode(output_ids,skip_special_tokens=True,clean_up_tokenization_spaces=False)
    print('TEXT:   ', article_text)
    print('SUMMARY:', summary)

In [6]:
test_records = read_gazeta_records("gazeta_test.txt")

In [7]:
HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))
model_name = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/730 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

/usr/local/lib/python3.8/dist-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Downloading:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

In [8]:
summurize(1)

TEXT:    Около 11 тысяч зрителей увидели все самое лучшее, что есть на сегодняшний день в культуре Бурятии. В Кремле выступил Бурятский государственный академический театр оперы и балета, Национальный цирк, Бурятский национальный театр песни и танца «Байкал», ставший победителем шоу «Танцуют все!» на телеканале «Россия», а также другие профессиональные и самодеятельные коллективы региона. Более 300 артистов из одного региона на главной сцене страны - похоже это рекорд России. Зрителям рассказали, что Республике Бурятия, чье население составляет 1 миллион человек, сохранилась и развивается культура десятков национальностей, включая русских, бурятов, староверов (семейских), эвенков. И все они были представлены в Москве. Как писали после шоу зрители в соцсетях: «А мы думали, что в Бурятии только буряты живут…». Для неподготовленного зрителя это вообще были вечера открытий. Например, когда еще в Кремлевском дворце выстраивалась очередь из желающих попасть на прием к врачам-пульсологам и ла

In [9]:
summurize(5)

TEXT:    Журналист Питер Суппли Бенсон датского издания Berlingske обратил внимание на усиление России в Арктике, а также на головной патрульный корабль ледового класса «Иван Папанин», который он назвал «новым кулаком» страны в регионе. В статье он указал, что новость о спуске боевого корабля на воду в Санкт-Петербурге получила международный резонанс. Автор материала отметил уникальность ледокола, оснащенного не только пушками, но и ракетами, которые могут быть применены для стрельбы и по другим кораблям, и по позициям противника на берегу. Длина » Ивана Папанина », согласно приведенным данным, составляет 114 м, а водоизмещение — 8,5 тыс. тонн. Ледокол оснастят крылатыми ракетами «Калибр», дальность которых может превышать 200 км. Планируется, что в арсенале этих ледоколов будет по восемь «Калибров». Учитывая эти обстоятельства, автор статьи выразил обеспокоенность в связи с тем, что Дания инвестирует в Арктику и Гренландию слишком мало средств и из-за этого отстает от России, которая 

In [10]:
summurize(10)

TEXT:    Спрос на бензин в России в 2019 году упал на 1%. Об этом сообщают «Известия», публикуя результаты исследования группы «Петромаркет» и агентства «Аналитика товарных рынков». Такое снижение спроса стало рекордом за последние годы. По мнению экспертов, тенденция объясняется не высокими ценами на бензин, а снижением реальных доходов населения. Автомобиль стал «довольно дорогим удовольствием», утверждают исследователи. Машина требует затрат на штрафы, техобслуживание и содержание. В результате россияне пересаживаются на такси и общественный транспорт. Некоторые автомобилисты переходят на более дешевый газ. При этом почти 50% россиян считают, что цены на бензин слишком высоки, и из-за этого машиной приходится пользоваться реже. Об этом свидетельствуют данные сентябрьского опроса ВЦИОМ. Среди других причин – дороговизна в обслуживании авто, выплата страховки и налогов (16%), попытки сэкономить средства (15%), пробки на дорогах (14%), забота об экологической обстановке (8% россиян) и 

In [11]:
summurize(15)

TEXT:    Росстандарт готовит арсенал для проверок автозаправочных станций на недолив топлива: проверяться будет не конкретная колонка, а ее работа как системы, в том числе ее связь с оператором за кассой. Контролеров при работе методом тайного покупателя вооружат скрытым прибором, который сможет «ловить» недолив на уровне 1%. Кроме того, новый ГОСТ защитит топливораздаточные колонки от приспособлений и вредоносного ПО, обманывающих расходомер. «Российская газета» пообщалась с главой Росстандарта Алексеем Абрамовым и выяснила, что в будущем с недоливом топлива на автозаправочных станциях будут бороться комплексно, используя большой арсенал инструментов. По словам Абрамова, недолив случается преимущественно из-за вторжения в программное обеспечение топливораздаточных колонок (ТРК). «Программа может быть настроена так, что на любом пистолете устанавливается конкретный процент недолива… Индикатор покажет вам 30 литров, но на самом деле в бак попало 29,7 литра, а если показывает 60 литров —

Предобученная модель очень хорошо справляется с суммаризацией